In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gp
%matplotlib inline

# Introduction

__Goal__:  Investigate a relationship between playground availability and household income in New York City.

__Data__: 

https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi

https://data.cityofnewyork.us/Environment/Directory-of-Playgrounds/59gn-q4ai

https://data.cityofnewyork.us/Recreation/Directory-of-Parks/79me-a7rs


#### Interesting links

https://comptroller.nyc.gov/reports/state-of-play-a-new-model-for-nyc-playgrounds/

https://www.nycgovparks.org/parks/brower-park/inspections/B012-03

https://www.nycgovparks.org/about/history/playgrounds


#### Zip codes datasets to identify NYC zipcodes vs NY state zipcodes

https://www.nycbynatives.com/nyc_info/new_york_city_zip_codes.php

https://journals.plos.org/plosone/article/file?type=supplementary&id=info:doi/10.1371/journal.pone.0194799.s001

# Terminology

__Playground vs. Park:__ To be clear, playgrounds and parks are separate entities. The main goal of this analysis is to investigate the relationship between __playgrounds__ and household income in NYC. _The Parks dataset is useful to provide a zipcode for the Playgrounds, however parks will not be considered playgrounds unless they are also present in the Playgrounds dataset._

A __playground__ is typically (outdoors) a large open space to play on, usually for children, while a __park__ is an area of land set aside for environment preservation and/or informal recreation ([unofficial source](https://wikidiff.com/park/playground))


# Ideas to look into, and how they relate to household income

 - #Playgrounds per zipcode
 - #Playgrounds per capita per zipcode
 - Accessibility of playgrounds
 - Of the playgrounds, which are actually attached to schools (therefore  less available)?
 - Sq footage of Playgrounds per zipcode (need dataset). 
 - Of the playgrounds, which are actually in decent condition? 
 - How has the Parks per capita changed over time?
 - can I predict park condition based on data?
 - can I predict median household income from park/playground condition, etc.

## Potential Applications
 - <font color=blue>__Does the creation of new playgrounds in a low-income zipcode lead to gentrification? Does it therefore act as a predictor for increasing household income? Increasing property values?__</font>
     - Ex: The High Line Park by Hudson Yards?
     - Maybe this question is better geared towards parks, not playgrounds


# Loading the Data

### Playgrounds Dataset

In [2]:
playgrounds = pd.read_json('../data/playgrounds/DPR_Playgrounds_001.json')
playgrounds.head()

,Accessible,Adaptive_Swing,Level,Location,Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon
0,Y,N,4.0,"Noble, Franklin, Milton Streets",American Playground,B001,B001,None,None,40.7288,-73.9579
1,Y,N,4.0,BAY PKWY & CROPSEY AVENUE,Bensonhurst Park,B007-01,B007,None,None,40.5969,-73.9998
2,Y,N,1.0,DUMONT AVE & BRISTOL ST,Betsy Head Memorial Playground,B008-03,B008,None,None,40.6645,-73.9118
3,Y,N,2.0,BROOKLYN AVE & PROSPECT PL,Brower Park,B012-02,B012,None,None,40.6735,-73.9438
4,Y,Y,2.0,BROOKLYN AVE & PROSPECT PL,Brower Park,B012-03,B012,None,None,40.6744,-73.9432


### Parks Dataset

In [3]:
parks = pd.read_json('../data/parks/DPR_Parks_001.json')
parks.head()

,Location,Name,Prop_ID,Zip
0,Franklin St. bet. Milton St. and Noble St.,American Playground,B001,11222
1,"E. 38 St., E. 38 St. bet. Ave. I and Ave. J",Amersfort Park,B002,11210
2,"Prospect Park W., 15 St.",Bartel-Pritchard Square,B003,11215
3,"Broadway, Stuyvesant Ave., Vernon Ave.",Beattie Square,B006,11221
4,Cropsey Ave. bet. 21 Ave. and Bay Pkwy.,Bensonhurst Park,B007,11214


### Tax Return Dataset

In [4]:
tax_returns = pd.read_csv('../data/income/16zpallagi.csv')
tax_returns_ny = tax_returns.loc[tax_returns['STATE']=='NY',:].reset_index(drop=True)
tax_returns_ny.head()


,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,36,NY,0,1,3445320,2355000,445210,595120,2114350,4860320,...,1713210,1659729,20,8,0,0,441650,387884,2729330,5662377
1,36,NY,0,2,2124110,1162700,441680,474330,1285810,3760990,...,1816240,5000576,0,0,0,0,290630,482940,1801120,4662872
2,36,NY,0,3,1297620,644300,399250,216270,829800,2372130,...,1257120,7572716,30,18,20,42,258810,585359,1013190,2932751
3,36,NY,0,4,825090,307120,392040,99830,554500,1705160,...,816050,7895883,0,0,0,0,183170,543754,615490,2209076
4,36,NY,0,5,1242910,293570,841180,82060,877120,3053270,...,1237140,23154728,16500,4382,3910,2224,343770,1668063,840920,4104126


### Cleaning Tax Return dataset

 - Need to flatten dataset - 1 row per zipcode will be easier to work with, but its going to become a very wide dataset
 - Need to use only zipcodes from NYC

In [5]:
tax_returns_ny = tax_returns_ny.groupby('zipcode').sum()
tax_returns_ny.drop(['STATEFIPS', 'agi_stub'], axis=1, inplace=True)

# EDA: Parks and Playgrounds Datasets

Prop_IDs are supposed to be a unique identifier for the property, and I expected there to be one per row in both the parks and playgrounds dataframes, however thats not the case for the playgrounds dataset. Will need to investigate

In [6]:
print('''Number of rows in Parks dataframe: {}
Number of unique Prop_IDs in Parks Dataframe: {}\n'''.format(
    len(parks), parks['Prop_ID'].nunique()))
print('''Number of rows in Playgrounds dataframe: {}
Number of unique Prop_IDs in Playgrounds Dataframe: {}\n'''.format(
    len(playgrounds),playgrounds['Prop_ID'].nunique()))

print('Parks dataset has {} Prop_IDs not listed in the playgrounds dataset'.format(
        len(set(parks.Prop_ID) - set(playgrounds.Prop_ID))))
print('Parks dataset has {} Prop_IDs not listed in the playgrounds dataset\n'.format(
        len(set(playgrounds.Prop_ID) - set(parks.Prop_ID))))


Number of rows in Parks dataframe: 1673
Number of unique Prop_IDs in Parks Dataframe: 1673

Number of rows in Playgrounds dataframe: 1269
Number of unique Prop_IDs in Playgrounds Dataframe: 870

Parks dataset has 834 Prop_IDs not listed in the playgrounds dataset
Parks dataset has 31 Prop_IDs not listed in the playgrounds dataset



In [7]:
playgrounds[playgrounds.Prop_ID.duplicated()]; # Remove ; to see all duplicates in playgrounds

__Many Playground_IDs are missing values__

In [8]:
print('{} playgrounds are missing Playground_IDs'.format(
    len(playgrounds[(playgrounds.Playground_ID.isna())])))

248 playgrounds are missing Playground_IDs


__Number of missing values per column in playgrounds dataset__ 

In [9]:
def count_missing_values(df):
    return len(df)-df.count()

In [10]:
count_missing_values(playgrounds)

Accessible           3
Adaptive_Swing      97
Level               95
Location             3
Name                 0
Playground_ID      248
Prop_ID              0
School_ID         1035
Status            1035
lat                 24
lon                 24
dtype: int64

For many of the above columns, missing values are to be expected. i.e. the school_id having a missing value simply means the playground is not associated with a school. 
    
    1.  Concerned about the 24 parks missing coordinates. Are the entries missing latitudes the same as the entries missing longitudes?
    
    2.  Are there any parks missing both coordinates and locations?


In [11]:
print('{} different playgrounds are missing coordinates'.format(
    len(playgrounds[(playgrounds.lon.isna()) | (playgrounds.lat.isna())])))

24 different playgrounds are missing coordinates


In [12]:
playgrounds[(playgrounds.lon.isna()) | (playgrounds.lat.isna())];

In [13]:
print('{} different playgrounds are missing both coordinates and Locations (see below)'.format(
    len(playgrounds[(playgrounds.lon.isna()) & (playgrounds.Location.isna())])))

playgrounds[(playgrounds.lon.isna()) & (playgrounds.Location.isna())]


2 different playgrounds are missing both coordinates and Locations (see below)


,Accessible,Adaptive_Swing,Level,Location,Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon
1239,Y,Y,1.0,None,Hunter's Point South Park,Q471,Q471,None,None,NaN,NaN
1252,Y,Y,2.0,None,Ranaqua Playground,X272,X272,None,None,NaN,NaN


The good news is that although both coordinates and locations are missing for 2 entries, they are identifiable and distinguishable via the Names (as well as the Prop_IDs and Playground_IDs)

# Joining the Playgrounds and Parks datasets

- Interested in Playgrounds, not parks (Park dataset is mainly needed for zipcodes). A Left join is ideal
- Consider each row in the Playgrounds Dataset as a separate playground. Join on the Prop_ID column to get a zipcode for each park. 
- Rename Locations and name for each dataset

In [14]:
playgrounds.rename(columns={'Name':'Playground_Name', 'Location':'Playground_Location'}, inplace=True)
parks.rename(columns={'Name':'Park_Name', 'Location':'Park_Location'}, inplace=True)
nyc_playgrounds = pd.merge(playgrounds, parks, how='left', on='Prop_ID')
nyc_playgrounds.head()

,Accessible,Adaptive_Swing,Level,Playground_Location,Playground_Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon,Park_Location,Park_Name,Zip
0,Y,N,4.0,"Noble, Franklin, Milton Streets",American Playground,B001,B001,None,None,40.7288,-73.9579,Franklin St. bet. Milton St. and Noble St.,American Playground,11222
1,Y,N,4.0,BAY PKWY & CROPSEY AVENUE,Bensonhurst Park,B007-01,B007,None,None,40.5969,-73.9998,Cropsey Ave. bet. 21 Ave. and Bay Pkwy.,Bensonhurst Park,11214
2,Y,N,1.0,DUMONT AVE & BRISTOL ST,Betsy Head Memorial Playground,B008-03,B008,None,None,40.6645,-73.9118,"Blake Ave., Dumont Ave., Livonia Ave. bet. Str...",Betsy Head Park,11212
3,Y,N,2.0,BROOKLYN AVE & PROSPECT PL,Brower Park,B012-02,B012,None,None,40.6735,-73.9438,"St. Mark's Ave., Park Pl. bet. Brooklyn Ave. a...",Brower Park,11213
4,Y,Y,2.0,BROOKLYN AVE & PROSPECT PL,Brower Park,B012-03,B012,None,None,40.6744,-73.9432,"St. Mark's Ave., Park Pl. bet. Brooklyn Ave. a...",Brower Park,11213


In [15]:
count_missing_values(nyc_playgrounds)

Accessible                3
Adaptive_Swing           97
Level                    95
Playground_Location       3
Playground_Name           0
Playground_ID           248
Prop_ID                   0
School_ID              1035
Status                 1035
lat                      24
lon                      24
Park_Location           260
Park_Name               260
Zip                     261
dtype: int64

## Consolidating Missing Zipcodes

In [16]:
print('{}% of the dataset is missing zipcodes'.format(round(
    len(nyc_playgrounds[nyc_playgrounds.Zip.isna()])/len(nyc_playgrounds)*100)))


21% of the dataset is missing zipcodes


It seems like many rows in the Playground_Location column contain a zipcode. Able to extract those zipcodes with regular expresion

In [17]:
def find_zip_from_address(df):
    df = df.copy() # prevent mutation to original dataframe
    # Searches Playground_Location series for presence of a zipcode on rows where zip is missing
    zipcodes = df[(df.Zip.isna())]['Playground_Location'].str.extract(r'(\d{5})')
    df.loc[(df.Zip.isna()),'Zip'] = zipcodes.values
    return df

    

In [18]:
nyc_playgrounds = find_zip_from_address(nyc_playgrounds)

In [19]:
print('{}% of the dataset is missing zipcodes now'.format(round(
    len(nyc_playgrounds[nyc_playgrounds.Zip.isna()])/len(nyc_playgrounds)*100)))

3% of the dataset is missing zipcodes now


Of the 36 remaining playgrounds with missing zipcodes, 9 are missing coordinates, which could be a problem later

In [20]:
count_missing_values(nyc_playgrounds[nyc_playgrounds.Zip.isna()])

Accessible              0
Adaptive_Swing          7
Level                   7
Playground_Location     0
Playground_Name         0
Playground_ID           9
Prop_ID                 0
School_ID              27
Status                 27
lat                     9
lon                     9
Park_Location          35
Park_Name              35
Zip                    36
dtype: int64

In [21]:
nyc_playgrounds[nyc_playgrounds.Zip.isna()];

### geopy

Luckily, there are a few options for getting around this problem.

Best practice would be to use a google maps python API, however due to API keys and the need for a billing account, it is not quite reproducible for the scope of this project. 

I alternatively found the __geopy__ [package](https://pypi.org/project/geopy/) which can convert coordinates to a zipcode, or a fuzzy address search (i.e. 'squibb park' or 'jay st nyc') to a zipcode. Install directions below

`pip install geopy`


In [22]:
from geopy.geocoders import Nominatim
import re

In [23]:
geolocator = Nominatim(user_agent="application")

In [24]:
def coord_to_zip(df, geolocator):
    df = df.copy()
    df2 = df.replace(np.NaN,-1)
    df['Zip'] = df2.apply(_coord_to_zip, axis=1,args=(geolocator,))
    return df

def _coord_to_zip(row, geolocator):
    if row['Zip'] != -1:#If zip is present, return zip as is
        return row['Zip']
    # Zip is missing:
    if row['lat'] != -1.0: #If coordinates are present
        coordinate_query = '{}, {}'.format(row['lat'], row['lon'])
        return geolocator.reverse(coordinate_query).raw['address']['postcode'][:5]
    else:
        return np.NaN
    

### Converting coordinates to zipcodes for null zipcode entries

In [25]:
nyc_playgrounds = coord_to_zip(nyc_playgrounds, geolocator)
count_missing_values(nyc_playgrounds)

Accessible                3
Adaptive_Swing           97
Level                    95
Playground_Location       3
Playground_Name           0
Playground_ID           248
Prop_ID                   0
School_ID              1035
Status                 1035
lat                      24
lon                      24
Park_Location           260
Park_Name               260
Zip                       9
dtype: int64

In [26]:
print('{}% of the dataset is missing zipcodes now'.format(round(
    len(nyc_playgrounds[nyc_playgrounds.Zip.isna()])/len(nyc_playgrounds)*100,2)))


0.71% of the dataset is missing zipcodes now


#### Dropping the remaining null zipcodes

There are still 9 playgrounds missing zipcodes. I found that I could not depend on geophy's address search to return the proper zipcode (__address__ search, __not__ coordinate search), so dropping the remaining 0.71% of entries is the more scalable solution in this case since it is such a low percentage.

If I were to use Google Maps API, I may be able to avoid this problem and create a scalable search base on park name, borough, and street addresses listed for those entries

In [27]:
nyc_playgrounds = nyc_playgrounds[nyc_playgrounds.Zip.notnull()].reset_index(drop=True)


In [28]:
# Count rows with null values
tax_returns_ny.isna().any(axis=1).sum()

0

# Playgrounds with multiple zipcodes listed

Is this a data entry error? 

- Approach 1: drop these columns
- Approach 2: Create a row for each zip. Maybe the playground is considered to be accessible to all of the listed zipcodes? Found this method to be problematic when manually checking zipcode distances.
- __Approach 3__ use the coordinates to determine 1 zipcode. Worried about a request limit from Geopy, but believe it might work. 

Worth testing Approach 1 vs. Approach 3 when looking for correlation between household income and playground availability, in case Geopy's coordinate search suffers from inaccuracies. `zipcode_testing.ipynb` was set up to empirically test geopy's coordinate accuracy, however due to request limits I have been unable to test with better confidence.

In [29]:
nyc_playgrounds.dropna(axis=0, subset=['Zip'], inplace=True)
nyc_playgrounds.reset_index(drop=True, inplace=True)
# sanity check that all zipcodes are atleast 5 digits
len(nyc_playgrounds[nyc_playgrounds.Zip.str.len() < 4])

0

In [30]:
print('Entries with multiple zipcodes listed: {}'.format(
    len(nyc_playgrounds[nyc_playgrounds.Zip.str.len() > 5])))

Entries with multiple zipcodes listed: 175


In [31]:
def clean_bad_zips(row):
    '''Filters out any zipcodes that are less than 5 digits from a list'''
    return [zipcode.strip() for zipcode in row if len(zipcode.strip()) >= 5]

# create a list transformation of the zipcodes present in the column
nyc_playgrounds.Zip = nyc_playgrounds.Zip.str.split(',')
# cleans out zipcodes that suffer from incorrect data entry
nyc_playgrounds.Zip = nyc_playgrounds.Zip.apply(clean_bad_zips)

### Some playgrounds list the same zip multiple times
Will use a __set__ transformation to eliminate these

In [32]:
# Notice the 5 identical zipcodes in the zip columns
nyc_playgrounds.loc[nyc_playgrounds['Playground_ID'] == 'B431']

,Accessible,Adaptive_Swing,Level,Playground_Location,Playground_Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon,Park_Location,Park_Name,Zip
985,Y,N,2.0,Main and Plymouth streets,Brooklyn Bridge Park,B431,B431,None,None,40.7041,-73.9902,"Furman St.,Water St. and John St. bet. Atlanti...",Brooklyn Bridge Park,"[11201, 11201, 11201, 11201, 11201]"


In [33]:
nyc_playgrounds.Zip = nyc_playgrounds.Zip.apply(set).apply(list)

In [34]:
# Now there is only one zipcode instead of 5 identical zipcodes listed
nyc_playgrounds.loc[nyc_playgrounds['Playground_ID'] == 'B431']

,Accessible,Adaptive_Swing,Level,Playground_Location,Playground_Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon,Park_Location,Park_Name,Zip
985,Y,N,2.0,Main and Plymouth streets,Brooklyn Bridge Park,B431,B431,None,None,40.7041,-73.9902,"Furman St.,Water St. and John St. bet. Atlanti...",Brooklyn Bridge Park,[11201]


### All entries with multiple zipcodes also have coordinates

Can use geopy to find the proper zipcodes

In [35]:
print('Entries with multiple zipcodes listed: {}'.format(
    len(nyc_playgrounds[nyc_playgrounds.Zip.apply(len) > 1])))

# All entries have coordinates
print('Entries with multiple zipcodes listed and coordinates: {}'.format(
    len(nyc_playgrounds[(nyc_playgrounds.Zip.apply(len) > 1) & 
                (nyc_playgrounds.lat.notnull())])))

Entries with multiple zipcodes listed: 165
Entries with multiple zipcodes listed and coordinates: 165


In [36]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
testing_df = nyc_playgrounds[(nyc_playgrounds.Zip.apply(len) > 1) & 
                (nyc_playgrounds.lat.notnull())]
testing_df.loc[:,'Borough'] = testing_df.Prop_ID.str.slice(0,1)
testing_df.loc[:,'counter'] = [1]*len(testing_df)
testing_df.groupby('Borough').count()['counter']

Borough
B    16
M    58
Q    37
R    10
X    44
Name: counter, dtype: int64

Disproportionate number of parks with missing zips from each borough. Dropping these could likely be influential

### Obtaining zipcodes from coordinates with Geopy

In [38]:
def _coord_to_zip2(row, geolocator):
    coordinate_query = '{}, {}'.format(row['lat'], row['lon'])
    if geolocator.reverse(coordinate_query).raw:
        try:
            zipcode = geolocator.reverse(coordinate_query).raw['address']['postcode'][:5]
            # additional check to ensure correct zipcode
            if len(zipcode) == 5 and int(zipcode) < 20000:
                return zipcode
            else:
                return np.NaN
        except:
            return np.NaN
    else:
        return np.NaN
    

In [39]:
nyc_playgrounds.loc[(nyc_playgrounds.Zip.apply(len)>1), 'zipcode_geopy'] \
    = nyc_playgrounds.loc[(nyc_playgrounds.Zip.apply(len)>1)].apply(
        _coord_to_zip2, args=(geolocator,), axis=1)
                                                                                                                                                   

In [40]:
multi_zips = nyc_playgrounds.loc[(nyc_playgrounds.Zip.apply(len) > 1),:]

In [41]:
multi_zips['zipcode_geopy'].isna().sum()

16

In [42]:
def zip_match(row):
    '''Returns a Zipcode from zipcode_geopy column if it matches 
    a zip in the list of zipcodes for a given entry'''
    A, B = set(row['Zip']), set([row['zipcode_geopy']])
    if len(A.intersection(B)) == 1:
        return row['zipcode_geopy']

In [43]:
print('{}% of zipcodes from geopys coordinate search matched a listed zipcode'.format(
    round(multi_zips.apply(zip_match, axis=1).notnull().sum()
    /len(multi_zips)*100,2)))
      

63.03% of zipcodes from geopys coordinate search matched a listed zipcode


This was a useful method to choose which zipcode to use from the list of zipcode for these playgrounds. I feel very confident using those zipcodes for those entries since they match up.

There are still playgrounds left with multiple zipcodes. Are these incorrect, or is geopy incorrect? It seems to be a mix of both.

This raises a larger data issue that should be addressed if there is more time. For now, I will arbitrarily choose the first zipcode in the list if there is no match between the list and geopy search

### dataframe transformation

In [44]:
def zip_match(row):
    '''Returns a Zipcode from zipcode_geopy column if it matches 
    a zip in the list of zipcodes for a given entry'''
    A, B = set(row['Zip']), set([row['zipcode_geopy']])
    if len(A.intersection(B)) == 1:
        return row['zipcode_geopy']
    else:
        return row['Zip'][0]

In [45]:
nyc_playgrounds.loc[(nyc_playgrounds.Zip.apply(len)>1), 'Zip'] \
    = nyc_playgrounds.loc[(nyc_playgrounds.Zip.apply(len)>1)].apply(
        zip_match, axis=1)

In [46]:
nyc_playgrounds.Zip.isna().sum()

0

In [47]:
nyc_playgrounds.Zip = nyc_playgrounds.Zip.apply(
    lambda x: x[0] if len(x) == 1 else x)

# Clean nyc_playgrounds dataframe

The goal is to have have the number of playgrounds in each zipcode

#### Create a Borough column

In [48]:
def _dummy(df, column):
    return pd.concat((df, pd.get_dummies(df[column], 
        prefix=column, drop_first=True)), axis=1).drop(column, axis=1)


In [49]:
nyc_playgrounds.Zip = nyc_playgrounds.Zip.astype('int64')

### Clean Accessible and Adaptive Swing columns

In [50]:
nyc_playgrounds['Adaptive_Swing'] = nyc_playgrounds['Adaptive_Swing'].replace({'':np.NaN})
nyc_playgrounds['Accessible'] = nyc_playgrounds.Accessible.replace({'N':0, 'Y':1})
nyc_playgrounds['Adaptive_Swing'] = nyc_playgrounds.Adaptive_Swing.replace({'N':0, 'Y':1})


In [51]:
nyc_playgrounds['Adaptive_Swing'].isna().sum()

97

In [52]:
nyc_playgrounds[(nyc_playgrounds['Accessible'].isna()) & 
                (nyc_playgrounds['Level'] > 0)]

,Accessible,Adaptive_Swing,Level,Playground_Location,Playground_Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon,Park_Location,Park_Name,Zip,zipcode_geopy


In [53]:
nyc_playgrounds['Accessible'].fillna(0, inplace=True)
nyc_playgrounds['Adaptive_Swing'].fillna(0, inplace=True) 

# Clean School_ID

Will create a boolean column to indicate if the playground is part of a school or not

In [54]:
nyc_playgrounds['School_ID'].fillna(0,inplace=True)
nyc_playgrounds['School'] = np.where(
    nyc_playgrounds['School_ID'] != 0, 1, 0)

# Clean Status Column

In [55]:
print('{}% of entries in the Status Column have null values\n'.format(
    round(nyc_playgrounds.Status.isna().sum()/
          len(nyc_playgrounds.Status)*100,1)))

81.4% of entries in the Status Column have null values



In [56]:
nyc_playgrounds.Status = nyc_playgrounds.Status.str.lower()
nyc_playgrounds.Status.unique()

array([None, 'open to the public', 'open to the public. north play',
       'temporarily closed: sca projec', 'open to the public: two playgr',
       'temporarily closed - sca proje', 'open to the public (further im',
       'open to the public on weekdays', 'open to the public: enter on w',
       'open weekends'], dtype=object)

The majority of values are null so this column may not be very useful down the line. However, it might be reasonable to clean this column into categories such as 'two playgrounds', 'closed', 'weekends', 'weekdays', and 'open to the public'. 

In [57]:
nyc_playgrounds.Status.fillna('None', inplace=True)
nyc_playgrounds.Status.loc[nyc_playgrounds['Status'].str.contains('closed')] = 'closed'
nyc_playgrounds.Status.loc[nyc_playgrounds['Status'].str.contains('weekend')] = 'weekends'
nyc_playgrounds.Status.loc[nyc_playgrounds['Status'].str.contains('weekday')] = 'weekdays'
nyc_playgrounds.Status.loc[nyc_playgrounds['Status'].str.contains('two playgr')] = 'two playgrounds'
nyc_playgrounds.Status.loc[nyc_playgrounds['Status'].str.contains('open to the public')] = 'open to the public'



### Status indicates 2 playgrounds

Important to investigate this, do the entries accurately reflect 2 playgrounds?

In [58]:
nyc_playgrounds[nyc_playgrounds['Status'] == 'two playgrounds']

,Accessible,Adaptive_Swing,Level,Playground_Location,Playground_Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon,Park_Location,Park_Name,Zip,zipcode_geopy,School
1015,1.0,0.0,4.0,"104 Sutter Avenue, Brooklyn, NY 11212",IS 392K,None,B,K356,two playgrounds,40.6658,-73.9178,NaN,NaN,11212,NaN,1
1112,1.0,0.0,4.0,"88 Woodbine Street, Brooklyn, NY 11221",PS 299K,None,B,K299,two playgrounds,40.6906,-73.9180,NaN,NaN,11221,NaN,1
1180,1.0,0.0,4.0,"216 Clawson Street, Staten Island, NY",PS 41R,None,R,R041,two playgrounds,40.5737,-74.1085,NaN,NaN,10306,NaN,1


In [59]:
# How is this reflected in the dataframe? with 1 entry or 2?
nyc_playgrounds[nyc_playgrounds.School_ID == 'R041']

,Accessible,Adaptive_Swing,Level,Playground_Location,Playground_Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon,Park_Location,Park_Name,Zip,zipcode_geopy,School
1180,1.0,0.0,4.0,"216 Clawson Street, Staten Island, NY",PS 41R,None,R,R041,two playgrounds,40.5737,-74.1085,NaN,NaN,10306,NaN,1


It seems like these entries with a status indicating 2 playgrounds should be transformed so that each zipcode will have an accurate representation of the number of playgrounds

However, after manually checking these addresses on Google maps satellite image view, it raises the question: 

Should two playground sets directly next to eachother on the same property count as two different playgrounds for the scope of this project? All 3 entries that include the string 'two playgr' in the Status column look similar to the image below:

<img src='img/img1.png'>

In terms of the psychological feel of this, would a local resident feel as if they had an extra playground accessible to them in their neighboorhood? Most likely not.

Considering only 3 playgrounds contain this string, and there are inconsistency issues with strings in this column, transforming these 3 entries to reflect 2 playgrounds each may not be a scalable approach long-term. For this reason I will not representing these entries as 2 playgrounds each in my dataset

# Investigating the relationship between Playgrounds at schools and Status column

Playgrounds located at schools are not open while the school is in session, which makes them less available to local residents. This is important to be represented in the dataframe

In [60]:
# Are there any Statuses when a School is not specified?
nyc_playgrounds[(nyc_playgrounds.School == 0) & 
                (nyc_playgrounds.Status != 'None')]

,Accessible,Adaptive_Swing,Level,Playground_Location,Playground_Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon,Park_Location,Park_Name,Zip,zipcode_geopy,School


In [61]:
nyc_playgrounds.Status[(nyc_playgrounds.School != 0) & 
                       (nyc_playgrounds.Status != 'None')].count()

234

Interestingly enough, the status column is only applicable to playgrounds located on school property

In [62]:
nyc_playgrounds = _dummy(nyc_playgrounds, 'Status')

# Clean Level column

To make this scalable, if accessible = 0, fillna(0), else fillna(1).
Filling null values with 1 does add some bias, however it is a very low percentage of the entries

In [63]:
# Sanity check. If park isn't accessible, its level shouldn't be > 0
len(nyc_playgrounds[(nyc_playgrounds['Level'] > 0.0) &
                (nyc_playgrounds.Accessible == 0.0)])

0

In [64]:
nyc_playgrounds.Level.loc[nyc_playgrounds['Accessible'] == 0.0] \
    = nyc_playgrounds.Level[nyc_playgrounds['Accessible'] == 0.0].fillna(0)


In [65]:
nyc_playgrounds[nyc_playgrounds['Level'].isna()]

,Accessible,Adaptive_Swing,Level,Playground_Location,Playground_Name,Playground_ID,Prop_ID,School_ID,lat,lon,Park_Location,Park_Name,Zip,zipcode_geopy,School,Status_closed,Status_open to the public,Status_two playgrounds,Status_weekdays,Status_weekends
1231,1.0,0.0,NaN,Kent Ave. between N. 9 St. and N. 12 St.,Bushwick Inlet Park,None,B529,0,40.7218,-73.9615,Kent Ave. bet. Quay St. and N 9 St.,Bushwick Inlet Park,11211,NaN,0,0,0,0,0,0
1245,1.0,0.0,NaN,"Murray Ave, Jarman Rd, Sylvester Ln, Abbot Rd",Fort Totten Playground,Q458,Q458,0,40.7941,-73.7780,Cross Island Pkwy. bet. Totten Ave. and 15 Rd.,Fort Totten Park,11359,NaN,0,0,0,0,0,0
1247,1.0,0.0,NaN,55th Dr & Borden Ave,Copernicus Triangle,QT05,QT05,0,40.7259,-73.9057,60 St. bet. Borden Ave. and 55 Dr.,Copernicus Triangle,11378,NaN,0,0,0,0,0,0


In [66]:
nyc_playgrounds['Level'].fillna(1, inplace=True)

In [67]:
nyc_playgrounds = _dummy(nyc_playgrounds, 'Level')

# Transform the dataset: grouping by zipcode

### Dropping incompatible columns

In [68]:
nyc_playgrounds.drop(
    nyc_playgrounds.select_dtypes(include=['object']).columns.tolist(), 
    axis=1,inplace=True)

In [69]:
nyc_playgrounds.drop(['lat', 'lon'], axis=1, inplace=True)

In [70]:
nyc_playgrounds['playground_count'] = [1]*len(nyc_playgrounds)

In [71]:
nyc_playgrounds = nyc_playgrounds.groupby('Zip').sum().reset_index()

In [72]:
nyc_playgrounds.head()

,Zip,Accessible,Adaptive_Swing,School,Status_closed,Status_open to the public,Status_two playgrounds,Status_weekdays,Status_weekends,Level_1.0,Level_2.0,Level_3.0,Level_4.0,playground_count
0,10001,2.0,1.0,0,0,0,0,0,0,0,0,0,2,3
1,10002,22.0,4.0,1,0,1,0,0,0,0,11,1,10,23
2,10003,4.0,2.0,0,0,0,0,0,0,0,2,2,0,4
3,10007,2.0,1.0,0,0,0,0,0,0,0,0,1,1,3
4,10009,7.0,1.0,0,0,0,0,0,0,0,0,1,6,7


# Merging playground data with shapefile data

https://data.cityofnewyork.us/widgets/i8iw-xf4u
https://data.cityofnewyork.us/City-Government/Parks-Zones/rjaj-zgq7

`pip install geopandas`

`pip install descartes`

In [73]:
zipcode_shape_data = gp.read_file(
    '../data/ZIP_CODE_040114 (3)/ZIP_CODE_040114.shp')\
    .to_crs(epsg=4326) # to_crs converts the coordinate system to familiar form


In [74]:
print('There are {} total zipcodes in NYC'.format(
    len(zipcode_shape_data['ZIPCODE'].unique())))

There are 248 total zipcodes in NYC


In [75]:
county_borough_map = {'Kings':'B', 
 'New York':'M', 
 'Richmond':'R',
'Queens':'Q',
'Bronx':'X'}

In [76]:
zipcode_shape_data['Borough'] = zipcode_shape_data['COUNTY'].map(county_borough_map)

In [77]:
zipcode_shape_data.head()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry,Borough
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,POLYGON ((-73.80584847647394 40.68290932644251...,Q
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,POLYGON ((-73.93739763139813 40.67972958925087...,B
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,POLYGON ((-73.90294132545438 40.67083977590012...,B
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,POLYGON ((-73.95797316043482 40.67065695897571...,B
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,POLYGON ((-73.97208109564257 40.65059658727614...,B


In [78]:
zip_df = zipcode_shape_data[['ZIPCODE', 'Borough', 'POPULATION', 'AREA', 'geometry']]


In [79]:
# looks for edge cases where keeping the first duplicate values would not
# identify the proper row (judged by incorrect borough)
zip_df[zip_df.ZIPCODE.duplicated(keep=False)].sort_values(by='ZIPCODE');

In [80]:
zip_df = zip_df[~((zip_df.ZIPCODE == '10463') & (zip_df.Borough == 'M'))]
zip_df = zip_df[~((zip_df.ZIPCODE == '11370') & (zip_df.Borough == 'X'))]
zip_df.drop_duplicates(subset='ZIPCODE', keep='first', inplace=True)
zip_df.reset_index(drop=True, inplace=True)
zip_df.ZIPCODE = zip_df.ZIPCODE.astype('int64')

### Merge Zipcode data with tax return data

In [81]:
tax_returns_df = pd.merge(zip_df, tax_returns_ny.reset_index(), how='inner', 
         left_on='ZIPCODE', right_on='zipcode').set_index('ZIPCODE')

# Merging data with Household Income data

In [111]:
df = pd.merge(nyc_playgrounds, tax_returns_df, how='right', left_on='Zip',
    right_on='ZIPCODE').set_index('zipcode').drop('Zip',axis=1)


In [113]:
df.head()

,Accessible,Adaptive_Swing,School,Status_closed,Status_open to the public,Status_two playgrounds,Status_weekdays,Status_weekends,Level_1.0,Level_2.0,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
zipcode,,,,,,,,,,,,,,,,,,,,,
10001,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12470,524842,1980,6213,1680,7081,3680,40079,9600,51137
10002,22.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,...,29030,356377,1280,2110,1070,2964,9000,35096,30040,81377
10003,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,26030,1566958,5110,18272,4730,30671,8100,105180,17200,133151
10007,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3180,767641,1400,8020,1240,19184,1170,28665,1730,67189
10009,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27250,450366,1650,2675,1320,4035,7040,36988,23570,66621


In [114]:
print('{} zips dont have parks at all'.format(
    len(df[df.isna().any(axis=1)])))
df[df.isna().any(axis=1)];

22 zips dont have parks at all


### dummy borough column

In [115]:
df['borough_name'] = df['Borough']
df = _dummy(df,'Borough')

# The Dataset is now relatively clean and ready to start working with

I have been following along in `src/utils.py` creating functions that replicate the work done in this notebook.

In [86]:
import os

In [91]:
os.getcwd()

'/Users/kylecaron/Desktop/Point72/notebooks'

In [ ]:
# os.chdir('notebooks/')

In [88]:
os.chdir('../')
from src.utils import import_clean_data
os.chdir('notebooks/')

In [97]:
df2 = import_clean_data()

### Checking the datasets are equivalent

In [98]:
set(df2.columns)- set(df.columns)

set()

In [99]:
set(df.columns)-set(df2.columns)

set()

In [100]:
len(df.columns)-len(df2.columns)

0

In [101]:
len(df2)-len(df)

0

# Exporting the Data for easy use

In [116]:
df2.to_csv('../data/tax_playgrounds.csv', index=True)

In [106]:
pd.read_csv('../data/tax_playgrounds.csv', index_col='zipcode');